# Bokeh Stocks Plot

The goal of this project is to make an interactive plot of the stock prices of Apple, Facebook, and Google, with a tooltip displaying the open, close, high, and low prices for each date. I am using the Python package [Bokeh](http://bokeh.pydata.org/en/latest/index.html) because it interacts well with Jupyter notebooks, allows for easy interactive plot creation, and can be used with nothing but Python (as opposed to something like D3.js).

This was done together with [Erin Elsbernd](https://github.com/erelsbernd).

## Setup

The main thing to note here is that `io.output_notebook()` allows interactive Bokeh plots to be displayed within a Jupyter notebook, rather than the standard behavior of creating a new HTML file and displaying it in a browser.

In [1]:
import bokeh.io as io
import bokeh.models as mod
import bokeh.plotting as plt
import numpy as np
import pandas as pd
import pandas_datareader.data as web

io.output_notebook()

Loading BokehJS ...

## Loading Data

Here we use the [pandas-datareader](https://pandas-datareader.readthedocs.io/en/latest/) package, which was until recently part of pandas, to easily read in financial data from Google finance. This outputs a rank-3 tensor of data as a pandas Panel object, but we turn this into a list of DataFrames to make the data easier to use with Bokeh. The start date was chosen to correspond to Facebook's IPO.

In [2]:
start_date = pd.to_datetime('2012-05-18')
stocks = ['AAPL', 'FB', 'GOOGL']

stock_dat = web.DataReader(stocks, 'google', start=start_date)
stocks_list = [stock_dat[:, :, ticker] for ticker in stocks]

## Creating the Plot

All of the plotted lines will share the same $x$-axis, corresponding to the dates since May 18, 2012. The $y$-axes are directly comparable: the price per share for each company at market close. This could obviously be modified to the opening price or whatever as desired.

Creating tooltips that work with each of the stocks separately can be tricky, but the approach used here is to create a single `Figure` object then iteratively populate it with elements corresponding to the different stocks. Because Bokeh line elements do not currently play nicely with its hover tool, we create a scatter plot in addition to the line for each stock; hovering over each point will give the tooltip.

The additional tools allow the user to zoom in to particular places of interest on the plot before using the hover tool to get more detailed information. They also allow the user to resize and save (portions of) the plot.

In [3]:
_tools_to_show = 'box_zoom, hover, pan, reset, resize, save'
_colors = ['black', 'blue', 'orange']

p = plt.figure(x_axis_type='datetime', tools=_tools_to_show,
              title='Apple, Facebook, and Google Share Prices',
              x_axis_label='Date',
              y_axis_label='Price ($)')
for i in range(3):
    p.scatter(x='Date', y='Close', color=_colors[i], size=1, source=stocks_list[i])
    hover = p.select({'type': mod.HoverTool})
    hover.tooltips = [('Open', '@Open'),
                      ('High', '@High'),
                      ('Low', '@Low'),
                      ('Close', '@Close')]
    p.line(x='Date', y='Close', color=_colors[i], source=stocks_list[i])
plt.show(p)